## Notebook to compute IVT from ERA5 wind and specific humidity
Water vapor transport can be diagnosed on a global scale by vertically integrating water vapor flux in a column,

$$Q_\lambda= \frac{1}{g}\int_{sfc}^{300hPa}qu\ dp$$

$$Q_\phi= \frac{1}{g}\int_{sfc}^{300hPa}qv\ dp$$

where $Q_\lambda$ is the zonal water vapor transport, $Q_\phi$ the meridional water vapor transport, $g$ the gravimetric constant, $q$ specific humidity, $u$ and $v$ the zonal and meridional component of total flow, respectively. IVT is then the magnitude of water vapor transport,

$$IVT=\sqrt{ Q_\lambda ^{\,\,2}\ +\ Q_\phi ^{\,\,2} }$$

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import cartopy
cartopy.config['pre_existing_data_dir']='/ihesp/shared/cartopy_features'
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
#-- Read data
path = r'/home/asblack/era5_files/'
filename_u = 'e5.oper.an.pl.128_131_u.ll025uv.1979010100_1979010123.nc'
filename_v = 'e5.oper.an.pl.128_132_v.ll025uv.1979010100_1979010123.nc'
filename_q = 'e5.oper.an.pl.128_133_q.ll025sc.1979010100_1979010123.nc'

da_u = xr.open_dataset(path + filename_u).isel(time=0, level=slice(17, 37)).U
da_v = xr.open_dataset(path + filename_v).isel(time=0, level=slice(17, 37)).V
da_q = xr.open_dataset(path + filename_q).isel(time=0, level=slice(17, 37)).Q

In [ ]:
#-- Reindex vertical levels
da_u = da_u.reindex(level=da_u.level[::-1])
da_v = da_v.reindex(level=da_v.level[::-1])
da_q = da_q.reindex(level=da_q.level[::-1])

In [ ]:
#-- Compute moisture flux
qu = da_u * da_q
qv = da_v * da_q

In [ ]:
#-- Integrate moisture flux over vertical levels
qu_int = integrate(qu, over_dim="level", x=(qu["level"] * 100) / 9.81)
qv_int = integrate(qv, over_dim="level", x=(qv["level"] * 100) / 9.81)

In [ ]:
#-- Compute IVT magnitude
IVT = np.sqrt(np.square(qu_int) + np.square(qv_int))

In [ ]:
#-- Plot

fig = plt.figure(figsize=(12, 10))
gsFig = plt.GridSpec(1, 1)
gsFig.update(left=0.05, right=0.95, bottom = 0.10, top = 0.90)
axFig = plt.subplot(gsFig[0, 0], projection=ccrs.PlateCarree(180))

image = IVT.plot.contourf(ax=axFig, levels = 11, cmap=plt.cm.Blues, add_colorbar=False, 
                                   extend='both', transform=ccrs.PlateCarree())
IVT.plot.contour(ax=axFig, levels = [200,201], colors='black',
                 transform=ccrs.PlateCarree())

#--Add coastlines
axFig.add_feature(cfeature.COASTLINE, edgecolor='black',linewidth=1, zorder=1)

#--Modify gridlines
gl = axFig.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.xlabels_top = False ; gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER ; gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 12} ; gl.ylabel_style = {'size': 12}

#--Fix colorbar
cbar = plt.colorbar(image, ax=axFig, fraction=0.02, pad=0.03, label='kg/m/s')

#--Labels
axFig.set_title('Integrated Water Vapor Transport',fontsize=18)

#--Save figure
#plt.savefig('./IVT.png')

### Next step: rotational and divergent components of water vapor transport

Total water vapor transport, $\mathbf Q$, can be divided into its rotational and divergent components ($\mathbf Q_R$ and $\mathbf Q_D$, respectively) as follows,

$$ \mathbf Q= \mathbf Q_R + \mathbf Q_D = \hat{k} \times \nabla \psi_{\mathbf Q} + \nabla \chi_{\mathbf Q} $$

where $\psi_{\mathbf Q}$ is the streamfunction of water vapor transport, and $\chi_{\mathbf Q}$ the potential function of water vapor transport. Therefore, the divergent component of vertically integrated water vapor transport is equivalent to the gradient of water vapor transport potential function. This leads to the following relationship,

$$ \nabla^2 \chi_{\mathbf Q} = \nabla \cdot \mathbf Q $$

so that we may solve the Poisson equation for $\chi_{\mathbf Q}$ using our derived values of $Q_\lambda$ and $Q_\phi$. We may then numerically differentiate the potential function of water vapor transport to solve for the divergent component of water vapor transport,

$$ \mathbf Q_D = \frac{\partial \chi_{\mathbf Q}}{\partial x}\hat{i} + \frac{\partial \chi_{\mathbf Q}}{\partial y}\hat{j} $$

In [ ]:
import windspharm as wsh

In [ ]:
psi_Q = stream_function(qu_int, qv_int, lat_name='latitude', lon_name='longitude')

## IVT with LLJ

In [ ]:
#-- Compute wind
wind_speed = np.sqrt( np.square(da_u) + np.square(da_v) )

In [ ]:
#-- Plot

fig = plt.figure(figsize=(12, 10))
gsFig = plt.GridSpec(1, 1)
gsFig.update(left=0.05, right=0.95, bottom = 0.10, top = 0.90)
axFig = plt.subplot(gsFig[0, 0], projection=ccrs.PlateCarree(180))

image = IVT.plot.contourf(ax=axFig, levels = 11, cmap=plt.cm.Blues, add_colorbar=False, 
                                   extend='both', transform=ccrs.PlateCarree())

IVT.plot.contour(ax=axFig, levels = [200,201], colors='darkgrey',
                 transform=ccrs.PlateCarree())

wind_speed.sel(level=925).plot.contour(ax=axFig, levels = [0,12,16,20,24],
                 transform=ccrs.PlateCarree())

#--Add coastlines
axFig.add_feature(cfeature.COASTLINE, edgecolor='black',linewidth=1, zorder=1)

#--Modify gridlines
gl = axFig.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.xlabels_top = False ; gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER ; gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 12} ; gl.ylabel_style = {'size': 12}

#--Fix colorbar
cbar = plt.colorbar(image, ax=axFig, fraction=0.02, pad=0.03, label='kg/m/s')

#--Labels
axFig.set_title('IVT, wind speed @ 925 hPa',fontsize=18)

### Slice over the extended southern plains regions

In [ ]:
#-- Slice data
global_offset = 360
IVT_slice = IVT.sel(latitude=slice(45,25), longitude=slice(global_offset-100,global_offset-80))
ws_slice = wind_speed.sel(latitude=slice(45,25), longitude=slice(global_offset-100,global_offset-80))

In [ ]:
#-- Plot

fig = plt.figure(figsize=(12, 10))
gsFig = plt.GridSpec(1, 1)
gsFig.update(left=0.05, right=0.95, bottom = 0.10, top = 0.90)
axFig = plt.subplot(gsFig[0, 0], projection=ccrs.PlateCarree(180))

image = IVT_slice.plot.contourf(ax=axFig, levels = 11, cmap=plt.cm.Blues, add_colorbar=False, 
                                   extend='both', transform=ccrs.PlateCarree())

IVT_slice.plot.contour(ax=axFig, levels = [200,201], colors='darkgrey',
                 transform=ccrs.PlateCarree())

jet = ws_slice.sel(level=925).plot.contour(ax=axFig, levels = [0,12,16,20,24],
                 transform=ccrs.PlateCarree())

#--Add coastlines
axFig.add_feature(cfeature.COASTLINE, edgecolor='black',linewidth=1, zorder=1)

#--Modify gridlines
gl = axFig.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.xlabels_top = False ; gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER ; gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 12} ; gl.ylabel_style = {'size': 12}

#--Fix colorbar
cbar = plt.colorbar(image, ax=axFig, fraction=0.02, pad=0.03, label='kg/m/s')

#--Labels
axFig.clabel(jet, inline=1, fontsize=10)
axFig.set_title('IVT, wind speed @ 925 hPa',fontsize=18)

#--Save figure
#plt.savefig('/home/bla390/control_run/images/IVT.png')

### Plot LR precip using tricontourf (no regridding)

In [ ]:
path = r'/ihesp/archive/LR/b.e13.B1950TRC5.ne30_g16.ihesp24_1950-2050.002/atm/hist/'
filename = 'b.e13.B1950TRC5.ne30_g16.ihesp24_1950-2050.002.cam.h4.1979-01-01-00000.nc'

ds = xr.open_dataset(path + filename).isel(time=0)

In [ ]:
fig = plt.figure(figsize=(10, 8))
gsFig = plt.GridSpec(1, 1)
gsFig.update(left=0.05, right=0.95, bottom = 0.10, top = 0.90)
axFig = plt.subplot(gsFig[0, 0], projection=ccrs.PlateCarree(180))

axFig.tricontourf(ds.lon, ds.lat, ds['PRECT'],
                  transform=ccrs.PlateCarree())

axFig.add_feature(cfeature.COASTLINE, edgecolor='black',linewidth=1, zorder=1)

#--Labels
axFig.set_title('Total precip (convective + large-scale)',fontsize=18)

In [ ]:
fig = plt.figure(figsize=(10, 8))
gsFig = plt.GridSpec(1, 1)
gsFig.update(left=0.05, right=0.95, bottom = 0.10, top = 0.90)
axFig = plt.subplot(gsFig[0, 0], projection=ccrs.PlateCarree(180))

axFig.tricontourf(ds.lon, ds.lat, ds['PRECC'],
                  transform=ccrs.PlateCarree())

axFig.add_feature(cfeature.COASTLINE, edgecolor='black',linewidth=1, zorder=1)

#--Labels
axFig.set_title('Convective precip only',fontsize=18)

### Plot HR precip using tricontourf (no regridding)

In [ ]:
path = 'r/ihesp/archive/HRMIP/B.E.13.B1950TRC5.ne120_t12.cesm-ihesp-1950-2050.shrflx.001/atm/hist/'
filename = 'B.E.13.B1950TRC5.ne120_t12.cesm-ihesp-1950-2050.shrflx.001.cam.h5.1980-01-01-03600.nc'

ds = xr.open_dataset(path + filename).isel(time=0)

In [ ]:
ds['lon'] = xr.where(ds.lon>180, ds.lon-360, ds.lon)

In [ ]:
fig = plt.figure(figsize=(10, 8))
gsFig = plt.GridSpec(1, 1)
gsFig.update(left=0.05, right=0.95, bottom = 0.10, top = 0.90)
axFig = plt.subplot(gsFig[0, 0], projection=ccrs.PlateCarree(180))

axFig.tricontourf(ds.lon, ds.lat, ds['PRECT'],
                  transform=ccrs.PlateCarree())

axFig.add_feature(cfeature.COASTLINE, edgecolor='black',linewidth=1, zorder=1)

## Notebook functions

In [ ]:
def integrate(da, over_dim, x=None, dx=None, method='trapz', cumulative=False, skipna=False):
    """ 
        Returns trapezoidal/rectangular integration along specified dimension 
        
        | Author: Dougie Squire
        | Date: 16/08/2018
        
        Parameters
        ----------
        da : xarray DataArray
            Array containing values to integrate
        over_dim : str
            Dimension to integrate
        x : xarray DataArray, optional
            Values to use for integrand. Must contain dimensions over_dim. If None, x is determined\
                    from the coords associated with over_dim
        dx : value, optional
            Integrand spacing used to compute the integral. If None, dx is determined from x
        method : str, optional
            Method of performing integral. Options are 'trapz' for trapezoidal integration, or 'rect'\
                    for rectangular integration
        cumulative : bool, optional
            If True, return the cumulative integral    
            
        Returns
        -------
        integral : xarray DataArray
            Array containing the integral along the specified dimension
            
        Examples
        --------
        >>> A = xr.DataArray(np.random.normal(size=(3,2)), coords=[('x', np.arange(3)), 
        ...                                                        ('y', np.arange(2))])
        >>> doppyo.utils.integrate(A, over_dim='x')
        <xarray.DataArray 'integral' (y: 2)>
        array([-0.20331 , -0.781251])
        Coordinates:
          * y        (y) int64 0 1
        See Also
        --------
        numpy.trapz
    """

    if x is None:
        x = da[over_dim]
    
    if len(x) == 1:
        if dx is None:
            raise ValueError('Must provide dx for integration along dimension with length 1')
        integral = (da * dx).drop(over_dim).squeeze()
    elif method == 'trapz':
        if dx is None:
            dx = x - x.shift(**{over_dim:1})
            dx = dx.fillna(0.0)

        if cumulative:
            integral = ((da.shift(**{over_dim:1}) + da) * dx / 2.0) \
                       .fillna(0.0) \
                       .cumsum(over_dim, skipna=skipna) 
        else:
            integral = ((da.shift(**{over_dim:1}) + da) * dx / 2.0) \
                       .fillna(0.0) \
                       .sum(over_dim, skipna=skipna) 
    elif method == 'rect':
        if dx is None:
            dx1 = x - x.shift(**{over_dim:1})
            dx2 = -(x - x.shift(**{over_dim:-1}))
            dx = dx1.combine_first(dx2)

        if cumulative:
            integral = (da * dx).cumsum(over_dim, skipna=skipna) 
        else:
            integral = (da * dx).sum(over_dim, skipna=skipna) 
    else:
        raise ValueError(f'{method} is not a recognised integration method')
    
    return integral

In [ ]:
def velocity_potential(u, v, lat_name=None, lon_name=None):
    import windspharm as wsh
    """ 
        Returns the velocity potential given fields of u and v
            
        | Author: Dougie Squire
        | Date: 11/07/2018
            
        Parameters
        ----------
        u : xarray DataArray
            Array containing fields of zonal velocity with at least coordinates latitude and longitude \
                    (following standard naming - see Notes)
        v : xarray DataArray
            Array containing fields of meridional velocity with at least coordinates latitude and \
                    longitude (following standard naming - see Notes)
        lat_name : str, optional
            Name of latitude coordinate. If None, doppyo will attempt to determine lat_name \
                    automatically
        lon_name : str, optional
            Name of longitude coordinate. If None, doppyo will attempt to determine lon_name \
                    automatically
                
        Returns
        -------
        velocity_potential : xarray DataArray
            Array containing values of velocity potential
                
        Examples
        --------
        >>> u = xr.DataArray(np.random.normal(size=(6,4)), 
        ...                  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90))])
        >>> v = xr.DataArray(np.random.normal(size=(6,4)), 
        ...                  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90))])
        >>> doppyo.diagnostic.velocity_potential(u, v)
        <xarray.DataArray 'velocity_potential' (lat: 6, lon: 4)>
        array([[  431486.75 ,   431486.75 ,   431486.75 ,   431486.75 ],
               [ -240990.94 , -3553409.   ,  -970673.56 ,  2341744.5  ],
               [ 3338223.5  ,  1497203.9  , -1723363.2  ,   117656.31 ],
               [ 1009613.5  ,  1571693.6  ,   326689.3  ,  -235390.69 ],
               [ -931064.8  ,  -124736.375, -2516887.8  , -3323216.   ],
               [-1526244.   , -1526244.   , -1526244.   , -1526244.   ]], dtype=float32)
        Coordinates:
          * lat      (lat) int64 75 45 15 -15 -45 -75
          * lon      (lon) int64 45 135 225 315
        Attributes:
            units:          m**2 s**-1
            standard_name:  atmosphere_horizontal_velocity_potential
            long_name:      velocity potential
    
        Notes
        -----------
        | All input array coordinates must follow standard naming (see ``doppyo.utils.get_lat_name()``, \
                ``doppyo.utils.get_lon_name()``, etc)
        | This function utilises the windspharm package, which is a wrapper on pyspharm, which is a \
                wrapper on SPHEREPACK. These packages require that the latitudinal and longitudinal grid \
                is regular or Gaussian.
        | These calculations are not yet dask-compatible.
    
        To Do
        - Make dask-compatible by either developing the windspharm package, or using a kernel approach
    """
    
    if lat_name is None:
        lat_name = get_lat_name(u)
    if lon_name is None:
        lon_name = get_lon_name(u)

    if not _equal_coords(u, v):
        raise ValueError('u and v coordinates must match')

    # Create a VectorWind instance -----
    w = wsh.xarray.VectorWind(u, v)

    # Compute the velocity potential -----
    return w.velocitypotential().rename('chi')

In [ ]:
def stream_function(u, v, lat_name=None, lon_name=None):
    import windspharm as wsh
    """ 
        Returns the stream function given fields of u and v
        
        | Author: Dougie Squire
        | Date: 11/07/2018
        
        Parameters
        ----------
        u : xarray DataArray
            Array containing fields of zonal velocity with at least coordinates latitude and longitude \
                    (following standard naming - see Notes)
        v : xarray DataArray
            Array containing fields of meridional velocity with at least coordinates latitude and \
                    longitude (following standard naming - see Notes)
        lat_name : str, optional
            Name of latitude coordinate. If None, doppyo will attempt to determine lat_name \
                    automatically
        lon_name : str, optional
            Name of longitude coordinate. If None, doppyo will attempt to determine lon_name \
                    automatically
            
        Returns
        -------
        stream_function : xarray DataArray
            Array containing values of stream function
            
        Examples
        --------
        >>> u = xr.DataArray(np.random.normal(size=(6,4)), 
        ...                  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90))])
        >>> v = xr.DataArray(np.random.normal(size=(6,4)), 
        ...                  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90))])
        >>> doppyo.diagnostic.stream_function(u, v)
        <xarray.DataArray 'psi' (lat: 6, lon: 4)>
        array([[ -690643.6 ,  -690643.6 ,  -690643.6 ,  -690643.6 ],
               [-2041977.8 , -1060127.  , -3052305.8 , -4034156.5 ],
               [ 4112389.2 ,  4630193.5 , -5212595.5 , -5730399.5 ],
               [  528500.75,  4670647.5 ,  2589393.  , -1552753.9 ],
               [-2686391.2 ,  -707369.25,  4204334.  ,  2225311.5 ],
               [ 1703481.9 ,  1703481.9 ,  1703481.9 ,  1703481.9 ]], dtype=float32)
        Coordinates:
          * lat      (lat) int64 75 45 15 -15 -45 -75
          * lon      (lon) int64 45 135 225 315
        Attributes:
            units:          m**2 s**-1
            standard_name:  atmosphere_horizontal_streamfunction
            long_name:      streamfunction
        
        Notes
        -----------
        | All input array coordinates must follow standard naming (see ``doppyo.utils.get_lat_name()``, \
                ``doppyo.utils.get_lon_name()``, etc)
        | This function utilises the windspharm package, which is a wrapper on pyspharm, which is a \
                wrapper on SPHEREPACK. These packages require that the latitudinal and longitudinal grid \
                is regular or Gaussian.
        | These calculations are not yet dask-compatible.
        
        To Do
        
        - Make dask-compatible by either developing the windspharm package, or using a kernel approach
    """

    if lat_name is None:
        lat_name = get_lat_name(u)
    if lon_name is None:
        lon_name = get_lon_name(u)

    if not _equal_coords(u, v):
        raise ValueError('u and v coordinates must match')
        
    # Create a VectorWind instance -----
    w = wsh.xarray.VectorWind(u, v)

    # Compute the streamfunction -----
    return w.stream_function().rename('psi')

In [ ]:
def get_lon_name(da):
    """ 
        Returns name of longitude dimension in input array
        
        | Author: Dougie Squire
        | Date: 03/03/2018
        
        Parameters
        ----------
        da : xarray DataArray
            Array with coordinate corresponding to longitude
        
        Returns
        -------
        name : str
            Name of dimension corresponding to longitude
        
        Examples
        --------
        >>> A = xr.DataArray(np.random.normal(size=(2,2,2,2,2)), 
        ...                  coords=[('lat', np.arange(2)), ('lon', np.arange(2)), 
        ...                          ('depth', np.arange(2)), ('level', np.arange(2))])
        >>> doppyo.utils.get_lon_name(A)
        'lon'
    """

    look_fors = ['lon', 'xt', 'xu']
    coords = []
    for look_for in look_fors:
        found = [look_for in dim for dim in da.coords]
        if sum(found) > 1:
            coords += list(compress(da.coords, found))
        elif sum(found) == 1:
            coords.append(list(compress(da.coords, found))[0])
    if coords == []:
        raise KeyError('Unable to determine longitude dimension')
    elif len(coords) == 1:
        return coords[0]
    else:
        return coords

In [ ]:
def get_lat_name(da):
    """ 
        Returns name of latitude dimension in input array
        
        | Author: Dougie Squire
        | Date: 03/03/2018
        
        Parameters
        ----------
        da : xarray DataArray
            Array with coordinate corresponding to latitude
        
        Returns
        -------
        name : str
            Name of dimension corresponding to latitude
        
        Examples
        --------
        >>> A = xr.DataArray(np.random.normal(size=(2,2,2,2,2)), 
        ...                  coords=[('lat', np.arange(2)), ('lon', np.arange(2)), 
        ...                          ('depth', np.arange(2)), ('level', np.arange(2))])
        >>> doppyo.utils.get_lat_name(A)
        'lat'
    """

    look_fors = ['lat', 'yt', 'yu']
    coords = []
    for look_for in look_fors:
        found = [look_for in dim for dim in da.coords]
        if sum(found) > 1:
            coords += list(compress(da.coords, found))
        elif sum(found) == 1:
            coords.append(list(compress(da.coords, found))[0])
    if coords == []:
        raise KeyError('Unable to determine latitude dimension')
    elif len(coords) == 1:
        return coords[0]
    else:
        return coords

In [ ]:
def _equal_coords(da_1, da_2):
    """ 
        Returns True if coordinates of da_1 and da_2 are equal (or flipped) 
        
        | Author: Dougie Squire
        | Date: 19/15/2018
        
        Parameters
        ----------
        da_1 : xarray DataArray
            First array to compare coordinates
        da_2 : xarray DataArray
            Second array to compare coordinates
            
        Returns
        -------
        equal : bool
            True if coordinates of da_1 and da_2 are equal (or flipped), False otherwise
            
        Examples
        --------
        >>> A = xr.DataArray(np.random.normal(size=(3,3)), coords=[('x', np.arange(3)), 
        ...                                                        ('y', np.arange(3))])
        >>> B = xr.DataArray(np.random.normal(size=(3,3)), coords=[('x', np.arange(3)), 
        ...                                                        ('y', np.arange(3))])
        >>> doppyo.utils._equal_coords(A,B)
        True
    """

    da1_coords = da_1.coords.to_dataset()
    da2_coords = da_2.coords.to_dataset()

    if da1_coords.equals(da2_coords):
        return True
    elif list(set(da_1.coords) - set(da_2.coords)) != []:
        return False
    else:
        # Check if coordinates are the same but reversed -----
        bool_list = [(da1_coords[coord].equals(da2_coords[coord])) | \
                     (da1_coords[coord].equals(da2_coords[coord] \
                                       .sel({coord:slice(None, None, -1)}))) \
                     for coord in da1_coords.coords]
        return np.all(bool_list)